In [8]:
import pandas as pd
import re

df = pd.read_csv("/var/colab/customer_support_tickets.csv")

print("Dataset Loaded Successfully")
print("Shape:", df.shape)

df['message'] = (
    df['Ticket Subject'].astype(str) + " " +
    df['Ticket Description'].astype(str)
)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_message'] = df['message'].apply(clean_text)

def classify_issue(text):
    if 'payment' in text or 'card' in text or 'billing' in text:
        return 'PAYMENT'
    elif 'login' in text or 'password' in text or 'signin' in text:
        return 'LOGIN'
    elif 'delivery' in text or 'late' in text or 'shipping' in text:
        return 'DELIVERY'
    elif 'refund' in text or 'return' in text:
        return 'REFUND'
    elif 'bug' in text or 'error' in text or 'issue' in text:
        return 'BUG'
    else:
        return 'GENERAL'

df['issue_type'] = df['clean_message'].apply(classify_issue)

def assign_priority(text):
    if 'urgent' in text or 'not working' in text or 'immediately' in text:
        return 'P0'
    elif 'error' in text or 'failed' in text:
        return 'P1'
    elif 'delay' in text or 'slow' in text:
        return 'P2'
    else:
        return 'P3'

df['priority'] = df['clean_message'].apply(assign_priority)

sla_map = {
    'P0': 4,
    'P1': 8,
    'P2': 24,
    'P3': 48
}

df['sla_hours'] = df['priority'].map(sla_map)

df['created_at'] = pd.Timestamp.now()
df['due_time'] = df['created_at'] + pd.to_timedelta(df['sla_hours'], unit='h')

manager_report = (
    df.groupby(['issue_type', 'priority'])
      .size()
      .reset_index(name='ticket_count')
)

df.to_csv("/var/colab/final_support_tickets.csv", index=False)
manager_report.to_csv("/var/colab/support_manager_report.csv", index=False)

print("\n PROJECT COMPLETED SUCCESSFULLY")
print("Generated files:")
print("1. final_support_tickets.csv")
print("2. support_manager_report.csv")


Dataset Loaded Successfully
Shape: (8469, 17)

 PROJECT COMPLETED SUCCESSFULLY
Generated files:
1. final_support_tickets.csv
2. support_manager_report.csv
